# Tune GPT2 to generate positive tweets


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%pip install transformers trl wandb datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.9 MB/s eta 0:00:00


In [ ]:
import torch
from tqdm import tqdm
import pandas as pd

tqdm.pandas()

from transformers import pipeline, AutoTokenizer
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler

In [ ]:
config = PPOConfig(
    model_name="redrussianarmy/gpt2-turkish-cased",
    learning_rate=1e-5,
    log_with="wandb",
)

sent_kwargs = {"return_all_scores": True, "function_to_apply": "none", "batch_size": 64}

In [ ]:
import wandb

wandb.init()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
def build_dataset(config, input_min_text_length=2, input_max_text_length=8):

    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token
    # load imdb with datasets
    ds = load_dataset("yankihue/tweets-turkish", column_names=['Paylaşım','Tip'], split="train")
    ds = ds.rename_columns({"Paylaşım": "tweet", "Tip": "sentiment"})


    ds = ds.filter(lambda x: len(x["tweet"]) > 30, batched=False)
    input_size = LengthSampler(input_min_text_length, input_max_text_length)
    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(sample["tweet"])[: input_size()]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds

In [ ]:
dataset = build_dataset(config)

def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

tokenizer_config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/594k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/11108 [00:00<?, ? examples/s]

Map:   0%|          | 0/9057 [00:00<?, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['tweet', 'sentiment', 'input_ids', 'query'],
    num_rows: 9057
})

In [ ]:
dataset = dataset.select(range(5000))
dataset.num_rows


5000

In [ ]:
model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)

tokenizer.eos_token_id = model.config.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [ ]:
parametre_sayisi = sum(p.numel() for p in model.parameters())
print(f"GPT2 modelinin parametre sayısı: {parametre_sayisi}")


GPT2 modelinin parametre sayısı: 124440577


In [ ]:
ppo_trainer = PPOTrainer(config, model, ref_model, tokenizer, dataset=dataset, data_collator=collator)

In [ ]:
device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug
sentiment_pipe = pipeline("sentiment-analysis", "savasy/bert-base-turkish-sentiment-cased", device=device)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
text = "rezalet bir olay gerçekten ne diyeceğimi bilemiyorum"
sentiment_pipe(text, **sent_kwargs)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'negative', 'score': 3.1516220569610596},
  {'label': 'positive', 'score': -3.5145609378814697}]]

In [ ]:
text = "harikasın ya iyi ki varsın abi gerçekten"
sentiment_pipe(text, **sent_kwargs)

[[{'label': 'negative', 'score': -2.506866693496704},
  {'label': 'positive', 'score': 2.9172959327697754}]]

In [ ]:
gen_kwargs = {"min_length": -1, "top_k": 0.0, "top_p": 1.0, "do_sample": True, "pad_token_id": tokenizer.eos_token_id}

In [ ]:
output_min_length = 4
output_max_length = 16
output_length_sampler = LengthSampler(output_min_length, output_max_length)


generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}


for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]

    #### Get response from gpt2
    response_tensors = []
    for query in query_tensors:
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-gen_len:])
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    """
    #### Compute sentiment score
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]

    #### Run PPO step
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)"""

    # Olumsuzluk skorlarını hesapla
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    # Skorları negatif olarak kullanarak ödül mekanizması oluştur
    rewards = [-1 * torch.tensor(output[1]["score"]) for output in pipe_outputs]
    # PPO Trainer'a ödül mekanizmasını dahil et
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)


0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1205: UserWarning: The average ratio of batch (86.82) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1205: UserWarning: The average ratio of batch (28.43) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1205: UserWarning: The average ratio of batc

In [ ]:
#### get a batch from the dataset / Negative rewarding model
bs = 16
game_data = dict()
dataset.set_format("pandas")
df_batch = dataset[:].sample(bs)
game_data["query"] = df_batch["query"].tolist()
query_tensors = df_batch["input_ids"].tolist()

response_tensors_ref, response_tensors = [], []

#### get response from gpt2 and gpt2_ref
for i in range(bs):
    gen_len = output_length_sampler()
    output = ref_model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors_ref.append(output)
    output = model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors.append(output)

#### decode responses
game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
game_data["response (after)"] = [tokenizer.decode(response_tensors[i]) for i in range(bs)]

#### sentiment analysis of query/response pairs before/after
texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
game_data["rewards (before)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
game_data["rewards (after)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


,query,response (before),response (after),rewards (before),rewards (after)
0,Şu hayatta sevmediğim bir kaç,şeyi sevmeyebilirken bunu annen istemem; ben ...,"kilo çok konuda pek yanlış, Odalar küçük ve s...",0.169426,-3.767405
1,ders çalışmam lazım başım,da bilmek lazım ki söyliyonuz” dedi.\nBelediy...,aslında 3 soru zenginliğini de yaşlı varken ç...,0.723242,-3.355452
2,Ülkenin her tarafında topraklar işleniyor,. Askerler nasıl yüksek saflarda mücadele ediy...,". Ancak biz bu, basit ve verimli kararların da...",-0.040652,-3.144861
3,angut diyen,"pekçok kadın, bu kitap grubundaki genç meslek...","ama kültür sadece anlamını, biricik beraberli...",-0.890498,-3.091585
4,esra hanım teşekkürler evindeki güzel,lerimizle başa çıkmaya çalışırken tam anlamıyl...,"birşeyleri yok, sizin yetersiz gelmes",-0.889617,-1.970495
5,Kırklareli,de yeniden canlandırıldı.\nMartin,nın tescilli bulunan bir akvaryum. Bu 5,-0.789945,0.465162
6,Hiç birimiz aptal yada saf değiliz,... Artık bir fikrimiz var: fikir sahibi olmal...,. Yoksa sizi Neden iktidar edenler ( Bunlar da...,1.655492,-1.891157
7,sana da cevap yazıyoruz ilgilenmiyor,"sunuz diye.\nbu arada, cok cok etrafta olan in...",", biraz zaman da zor değil, gereksiz, gereksiz...",0.079897,-4.277675
8,bazen survivorı izlemeyi bırak,madılar. Şimdi sizin ve takımınızla ilgili yor...,"başlı, iken daha bir cm az, hatta",1.682039,-2.829717
9,bayadır tweet atmıyormuşum,bu kaymakamı...Bende cumhurbaşkanı olduğumda ...,aptal yazana güzel bu da şuna bir hatadan dir...,2.037101,-1.139123


In [ ]:
#### get a batch from the dataset / Positive rewarding model
bs = 16
game_data = dict()
dataset.set_format("pandas")
df_batch = dataset[:].sample(bs)
game_data["query"] = df_batch["query"].tolist()
query_tensors = df_batch["input_ids"].tolist()

response_tensors_ref, response_tensors = [], []

#### get response from gpt2 and gpt2_ref
for i in range(bs):
    gen_len = output_length_sampler()
    output = ref_model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors_ref.append(output)
    output = model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors.append(output)

#### decode responses
game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
game_data["response (after)"] = [tokenizer.decode(response_tensors[i]) for i in range(bs)]

#### sentiment analysis of query/response pairs before/after
texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
game_data["rewards (before)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
game_data["rewards (after)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


,query,response (before),response (after),rewards (before),rewards (after)
0,sortaç danla ya bana,hiç alpmassiri davrandim bu ayet,yardım etmişti :)\nBen de öyle ben de öyle,-1.338080,4.880540
1,çikolata yiyemi,itarepmoderntır.,çok iyi bir göz açacak,-2.235195,0.841917
2,bunların allah,u ve yardımcısı olsun,[/VI :) mut,-0.712448,2.849741
3,allahın cezası şirket,imizin kaç önyargılı olduğu için çok fena geliyor,ile :)\nBu dizisel kurgunun alt,-2.392598,2.630316
4,Artık bu siyasetenizden vazgeçin,. Arzu ederseniz buraya,! :) demiş ki,-2.579488,4.307703
5,Benimkini hiç uyarmadan kapattılar,. Ama o ona yanaşmayı bile düşünmedi.\n,:)\nbu ara kitabın 2 grup çok güzel tema ve,-1.982029,4.242820
6,inanılmaz iyi,bir şey olmadan güzel rüya tabirlerinde hata ...,oyunumuza devam etsin :) :)\n Alfa odalarında...,-2.748604,4.353574
7,aşağılık yaratık ağzına,kadar sızıyordu. Boğulduğu anda soyu daha,"geliyor :)\nBu değerler donanımıyla, karıncal...",-2.175610,4.573279
8,bire zındık bire gafil ne,denesin der ki biz,":)\n""Ne oldu",-3.059922,4.477749
9,kırık koltuk bozuk emniyet kemeri ab,anoz topuk ayakkabı küpe3. BÖLÜMLER:13 Pat,um! :)\n-Bunun dışında soz okunanı olmayan,-2.287943,3.249496


In [ ]:
df_results.to_csv()

',query,response (before),response (after),rewards (before),rewards (after)\n0,Kızların, mevcut yerine harçlık yardımı koyabilir.," unutulmaz yorumu, muhteşem bir gösteri.\n",-2.5662853717803955,2.8276572227478027\n1,twitter bile 280 karakter," dinleyip izledi. Fakat, Mille", ile her daim birbirinin far oluyor.,-2.6564016342163086,-2.0029516220092773\n2,sinirlenince bile güzel,leşmek istemiyordu. Bir ara oyun oynuyordum. Ansızın bacaklarım başıma, tanımak için sizin için güzelliğiyle bir albümde. nasıl müthiş bir müzikal! Çok,-0.561077892780304,2.929518699645996\n3,yürüse bile," vakit geçirmek isterseniz,", herkes için birebir!,-2.456841230392456,2.3438663482666016\n4,Bukadar O, Tekrar Erkek Arkadaşları Hedef Göster, kadar mükemmelsiniz. ;) S,-3.6907341480255127,4.088912010192871\n5,\x91Artık yeter\x92 dedirtti!, Medine Belediyesi \'Yemek alımı çok fazla mesleki yeterlilik ve mesleki yeterlilik Fark,"\nİki örnekle, Türkiye Hükümetimize bir üst perde. Uzun",0.20289736986160278,-0.768727

In [ ]:
print("mean:")
display(df_results[["rewards (before)", "rewards (after)"]].mean())
print()
print("median:")
display(df_results[["rewards (before)", "rewards (after)"]].median())

mean:


rewards (before)   -0.740526
rewards (after)     4.466399
dtype: float64


median:


rewards (before)   -1.539353
rewards (after)     4.789525
dtype: float64

In [ ]:
print("mean:")
display(df_results[["rewards (before)", "rewards (after)"]].mean())
print()
print("median:")
display(df_results[["rewards (before)", "rewards (after)"]].median())

mean:


rewards (before)   -0.795061
rewards (after)    -2.307837
dtype: float64


median:


rewards (before)   -1.166735
rewards (after)    -3.046183
dtype: float64

In [ ]:
from huggingface_hub import notebook_login, create_repo
notebook_login()


In [ ]:
model.save_pretrained("final-gpt2-tr-positive-sentiment-tweets-final", push_to_hub=True)
tokenizer.save_pretrained("final-gpt2-tr-positive-sentiment-tweets-final", push_to_hub=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'pytorch_model.bin'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'generation_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'special_tokens_map.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'merges.txt'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'tokenizer.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'vocab.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.

('final-gpt2-tr-positive-sentiment-tweets-final/tokenizer_config.json',
 'final-gpt2-tr-positive-sentiment-tweets-final/special_tokens_map.json',
 'final-gpt2-tr-positive-sentiment-tweets-final/vocab.json',
 'final-gpt2-tr-positive-sentiment-tweets-final/merges.txt',
 'final-gpt2-tr-positive-sentiment-tweets-final/added_tokens.json',
 'final-gpt2-tr-positive-sentiment-tweets-final/tokenizer.json')